# Lab 4 - Hybrid Search of title embeddings and full-text fields in Opensearch

In [1]:
require 'opensearch'
require 'transformers-rb'
require 'tqdm'
require 'date'
require 'json'
require 'polars-df'

true

In [2]:
# https://github.com/opensearch-project/opensearch-ruby
host = 'ruby-opensearch-node'
port = 9200

$client = OpenSearch::Client.new(hosts: [{ host: host, port: port }])
info = $client.info
puts "Welcome to #{info['version']['distribution']} #{info['version']['number']}!"

Welcome to opensearch 2.11.0!


## Use the same model and method to get the query embedding, with some defaults changed
Remember, the model is `intfloat/e5-small-v2` and we need to prefix any query with 'query:'

In [3]:
# The E5 models expect 'query:' and 'passage:' prefixes
$model = Transformers.pipeline("embedding", 'intfloat/e5-small-v2')

def get_embeddings(texts, prefix: "query: ", progress: false)
  # puts texts
  texts = [texts] unless texts.is_a?(Array)
  total = texts.length
  embeddings = []
  
  texts.each_with_index do |text, i|
    prefixed_text = "#{prefix}#{text}"
    embedding = $model.(prefixed_text)
    embeddings << embedding
    
    if progress
        percent = ((i + 1).to_f / total * 100).to_i
        print "\rProcessing embeddings: #{percent}% (#{i + 1}/#{total})"
    end
  end
  if progress
      print "\nDone!\n"
  end
  embeddings
end

:get_embeddings

## Define different query types
Here we define Opensearch query bodies for:
 - BM25
 - KNN
 - Hybrid

In [4]:
def get_bm25_body(querystring)
    {
      "query" => {
        "bool" => {
          "should" => [
            {
              "multi_match" => {
                "query" => querystring,
                "type" => "cross_fields",
                "fields" => ["description"],
                "boost" => 1.0
              }
            },
            {
              "multi_match" => {
                "query" => querystring,
                "type" => "cross_fields",
                "fields" => ["title"],
                "boost" => 1.1
              }
            },
            {
              "multi_match" => {
                "query" => querystring,
                "type" => "cross_fields",
                "fields" => ["title_exactish"],
                "boost" => 1.2
              }
            }
          ]
        }
      },
      "_source" => {"exclude" => ["title_embedding"]}
    }
end

:get_bm25_body

In [5]:
def get_knn_body(querystring)
    embeddings = get_embeddings(querystring)
    {
      "query" => {
        "bool" => {
          "should" => [
            {
              "knn" => {
                "title_embedding" => {
                  "vector" => embeddings[0],
                  "k" => 20
                }
              }
            }
          ]
        }
      },
      "_source" => {"exclude" => ["title_embedding"]}
    }
end

:get_knn_body

## Hybrid Query

In [6]:
def get_hybrid_body(querystring)
    embeddings = get_embeddings(querystring)
    {
      "query" => {
        "hybrid" => {
          "queries" => [
            {
              "bool" => {
                "should" => [
                  {
                    "multi_match" => {
                      "query" => querystring,
                      "type" => "cross_fields",
                      "fields" => ["description"],
                      "boost" => 1.0
                    }
                  },
                  {
                    "multi_match" => {
                      "query" => querystring,
                      "type" => "cross_fields",
                      "fields" => ["title"],
                      "boost" => 1.1
                    }
                  },
                  {
                    "multi_match" => {
                      "query" => querystring,
                      "type" => "cross_fields",
                      "fields" => ["title_exactish"],
                      "boost" => 1.2
                    }
                  }
                ]
              }        
            },
            {
              "knn" => {
                "title_embedding" => {
                  "vector" => embeddings[0],
                  "k" => 100
                }
              }
            }
          ]
        }
      },
      "_source" => {"exclude" => ["title_embedding"]}
    }
end

:get_hybrid_body

In [19]:
def serps(querystring, resp, k: 5, show: true)
    # this doesn't match the value in the other notebook.
    count = resp["hits"]["total"]["value"]
    results = resp["hits"]["hits"]
    
    html_str = "<h4>Showing #{count} Results for <em>#{querystring}</em></h4><ol>"
    
    results[0...k].each do |result|
        score = result["_score"]
        title = result["_source"]["title"] || "No title"
        url = result["_source"]["url"] || "No URL"
        description = result["_source"]["description"]
        text = result["_source"]["text"] || ""
        snippet = description || "#{text[0...140]}..."
        
        html_str += "<li><b>#{title}</b>(#{score})<br>#{snippet}<br>"
        html_str += "<span style=\"font-size:0.8em\"><a href=\"#{url}\">#{url}</a></span></li>"
    end
    
    html_str += "</ol>"
    
    if show
        IRuby.display(IRuby.html(html_str))
    else
        html_str
    end
end

:serps

In [8]:
def make_normalization_pipeline(name, bm25_weight: 0.5, knn_weight: 0.5)
  body = {
    "description" => "Post processor for hybrid search with bm25=#{bm25_weight} and knn=#{knn_weight}",
    "phase_results_processors" => [
      {
        "normalization-processor" => {
          "normalization" => {
            "technique" => "min_max"
          },
          "combination" => {
            "technique" => "arithmetic_mean",
            "parameters" => {
              "weights" => [
                bm25_weight,
                knn_weight
              ]
            }
          }
        }
      }
    ]
  }
  
  # Changed to use positional parameters instead of named parameters
  resp = $client.transport.perform_request(
    "PUT",                              # method
    "/_search/pipeline/#{name}",        # path
    {},                                 # params
    body                                # body
  )
end

:make_normalization_pipeline

In [9]:
make_normalization_pipeline("nlp-search-pipeline-equal", bm25_weight: 0.5, knn_weight: 0.5)
make_normalization_pipeline("nlp-search-pipeline-bm25-heavy", bm25_weight: 0.6, knn_weight: 0.4)
make_normalization_pipeline("nlp-search-pipeline-knn-heavy", bm25_weight: 0.4, knn_weight: 0.6)

#<OpenSearch::Transport::Transport::Response:0x0000004039be6620 @status=200, @body={"acknowledged"=>true}, @headers={"content-type"=>"application/json; charset=UTF-8", "content-encoding"=>"gzip", "content-length"=>"47"}>

In [20]:
def search(querystring, body, pipeline: "nlp-search-pipeline-equal", show: true)
  resp = $client.search({
    index: "ai-search",
    body: body,
    search_pipeline: pipeline  # Moved from params into main options hash
  })
  serps(querystring, resp) if show
  resp
end

def search_bm25(querystring, show: true)
    body = get_bm25_body(querystring)
    search(querystring, body, show: show)
end

def search_knn(querystring, show: true)
    body = get_knn_body(querystring)
    search(querystring, body, show: show)
end

# def search_hybrid(querystring, pipeline: "nlp-search-pipeline-equal", show: true)
#     body = get_hybrid_body(querystring)
#     search(querystring, body, pipeline: pipeline, show: show)
# end

def search_hybrid(querystring, pipeline: "nlp-search-pipeline-equal", show: true)
  body = get_hybrid_body(querystring)
  search(querystring, body, pipeline: pipeline, show: show)
end

:search_hybrid

In [23]:
resp = search_hybrid("crypto scandal")
nil

"<h4>Showing 10 Results for <em>crypto scandal</em></h4><ol><li><b>AP Explains: Why are crypto exchanges vulnerable to hacks?</b>(0.5)<br>AP Explains: Why are crypto exchanges vulnerable to hacks?<br><span style=\"font-size:0.8em\"><a href=\"https://www.taiwannews.com.tw/en/news/3353987\">https://www.taiwannews.com.tw/en/news/3353987</a></span></li><li><b>Bitcoin, other cryptocurrencies plunge on wave of bad news</b>(0.5)<br>Google said it will ban cryptocurrency-related ads, the IMF called for global regulation and members of Congress slammed them at a hearing Wednesday.<br><span style=\"font-size:0.8em\"><a href=\"https://www.nbcnews.com/tech/tech-news/bitcoin-other-cryptocurrencies-plunge-wave-bad-news-n856821?cid=public-rss_20180320\">https://www.nbcnews.com/tech/tech-news/bitcoin-other-cryptocurrencies-plunge-wave-bad-news-n856821?cid=public-rss_20180320</a></span></li><li><b>‘Fraud.’ ‘More than a fad.’ The words Wall Street CEOs are using to describe bitcoin</b>(0.4570444)<br>Seems not a day goes by without some Wall Street VIP or financial luminary deciding it’s time for them to weigh in on bitcoin and cryptocurrencies. Here’s a roundup of the latest high-profile statements.<br><span style=\"font-size:0.8em\"><a href=\"http://www.marketwatch.com/story/fraud-or-the-future-how-bitcoin-is-being-described-by-billionaires-and-wall-street-ceos-2017-10-05?siteid=rss&rss=1\">http://www.marketwatch.com/story/fraud-or-the-future-how-bitcoin-is-being-described-by-billionaires-and-wall-street-ceos-2017-10-05?siteid=rss&rss=1</a></span></li><li><b>First U.S. cryptocurrency ban outlaws Venezuela’s ‘Petro’ coin</b>(0.45533127)<br>The U.S. executive branch has issued an executive order prohibiting the use or purchase of Venezuela's Petro cryptocurrency.<br><span style=\"font-size:0.8em\"><a href=\"https://www.digitaltrends.com/computing/us-bans-petro-cryptocurrency/\">https://www.digitaltrends.com/computing/us-bans-petro-cryptocurrency/</a></span></li><li><b>Trump bans US use of Venezuelan cryptocurrency - WAFB 9 News Baton Rouge, Louisiana News, Weather, Sports</b>(0.45388967)<br><br><span style=\"font-size:0.8em\"><a href=\"http://www.wafb.com/story/37757576/trump-bans-us-use-of-venezuelan-cryptocurrency\">http://www.wafb.com/story/37757576/trump-bans-us-use-of-venezuelan-cryptocurrency</a></span></li></ol>"

In [24]:
resp = search_bm25("crypto scandal")
nil

"<h4>Showing 164 Results for <em>crypto scandal</em></h4><ol><li><b>AP Explains: Why are crypto exchanges vulnerable to hacks?</b>(29.908903)<br>AP Explains: Why are crypto exchanges vulnerable to hacks?<br><span style=\"font-size:0.8em\"><a href=\"https://www.taiwannews.com.tw/en/news/3353987\">https://www.taiwannews.com.tw/en/news/3353987</a></span></li><li><b>Bitcoin loses more than half its value amid crypto crash</b>(26.140495)<br>After stratospheric growth, Bitcoin crashed below $8,000 for the first time in months amid a continuing global crypto rout.<br><span style=\"font-size:0.8em\"><a href=\"https://www.nbcnews.com/tech/internet/bitcoin-loses-more-half-its-value-amid-crypto-crash-n844056?cid=public-rss_20180202\">https://www.nbcnews.com/tech/internet/bitcoin-loses-more-half-its-value-amid-crypto-crash-n844056?cid=public-rss_20180202</a></span></li><li><b>Bitcoin loses more than half its value amid crypto crash</b>(26.140495)<br>After stratospheric growth, Bitcoin crashed below $8,000 for the first time in months amid a continuing global crypto rout.<br><span style=\"font-size:0.8em\"><a href=\"https://www.nbcnews.com/tech/internet/bitcoin-loses-more-half-its-value-amid-crypto-crash-n844056?cid=public-rss_20180204\">https://www.nbcnews.com/tech/internet/bitcoin-loses-more-half-its-value-amid-crypto-crash-n844056?cid=public-rss_20180204</a></span></li><li><b>Bitcoin loses more than half its value amid crypto crash</b>(26.140495)<br>After stratospheric growth, Bitcoin crashed below $8,000 for the first time in months amid a continuing global crypto rout.<br><span style=\"font-size:0.8em\"><a href=\"https://www.nbcnews.com/tech/internet/bitcoin-loses-more-half-its-value-amid-crypto-crash-n844056?cid=public-rss_20180203\">https://www.nbcnews.com/tech/internet/bitcoin-loses-more-half-its-value-amid-crypto-crash-n844056?cid=public-rss_20180203</a></span></li><li><b>Timeline: Irish Olympic ticket-sale scandal</b>(24.739798)<br>Timeline of events in the Rio Olympics illegal ticket-sale scandal.<br><span style=\"font-size:0.8em\"><a href=\"http://www.bbc.com/news/world-europe-37156930\">http://www.bbc.com/news/world-europe-37156930</a></span></li></ol>"

In [25]:
resp = search_knn("crypto scandal")
nil

"<h4>Showing 460 Results for <em>crypto scandal</em></h4><ol><li><b>Bitcoin, other cryptocurrencies plunge on wave of bad news</b>(0.8700277)<br>Google said it will ban cryptocurrency-related ads, the IMF called for global regulation and members of Congress slammed them at a hearing Wednesday.<br><span style=\"font-size:0.8em\"><a href=\"https://www.nbcnews.com/tech/tech-news/bitcoin-other-cryptocurrencies-plunge-wave-bad-news-n856821?cid=public-rss_20180320\">https://www.nbcnews.com/tech/tech-news/bitcoin-other-cryptocurrencies-plunge-wave-bad-news-n856821?cid=public-rss_20180320</a></span></li><li><b>‘Fraud.’ ‘More than a fad.’ The words Wall Street CEOs are using to describe bitcoin</b>(0.8627867)<br>Seems not a day goes by without some Wall Street VIP or financial luminary deciding it’s time for them to weigh in on bitcoin and cryptocurrencies. Here’s a roundup of the latest high-profile statements.<br><span style=\"font-size:0.8em\"><a href=\"http://www.marketwatch.com/story/fraud-or-the-future-how-bitcoin-is-being-described-by-billionaires-and-wall-street-ceos-2017-10-05?siteid=rss&rss=1\">http://www.marketwatch.com/story/fraud-or-the-future-how-bitcoin-is-being-described-by-billionaires-and-wall-street-ceos-2017-10-05?siteid=rss&rss=1</a></span></li><li><b>First U.S. cryptocurrency ban outlaws Venezuela’s ‘Petro’ coin</b>(0.8624979)<br>The U.S. executive branch has issued an executive order prohibiting the use or purchase of Venezuela's Petro cryptocurrency.<br><span style=\"font-size:0.8em\"><a href=\"https://www.digitaltrends.com/computing/us-bans-petro-cryptocurrency/\">https://www.digitaltrends.com/computing/us-bans-petro-cryptocurrency/</a></span></li><li><b>Trump bans US use of Venezuelan cryptocurrency - WAFB 9 News Baton Rouge, Louisiana News, Weather, Sports</b>(0.8622549)<br><br><span style=\"font-size:0.8em\"><a href=\"http://www.wafb.com/story/37757576/trump-bans-us-use-of-venezuelan-cryptocurrency\">http://www.wafb.com/story/37757576/trump-bans-us-use-of-venezuelan-cryptocurrency</a></span></li><li><b>Equifax executive hit with insider trading charge for selling stock before data breach went public</b>(0.86135316)<br>A former executive for Equifax is facing criminal and anti-fraud charges after selling stock options before the company went public with the megabreach.<br><span style=\"font-size:0.8em\"><a href=\"https://www.nbcnews.com/business/business-news/equifax-executive-hit-insider-trading-charge-selling-stock-data-breach-n856601?cid=public-rss_20180320\">https://www.nbcnews.com/business/business-news/equifax-executive-hit-insider-trading-charge-selling-stock-data-breach-n856601?cid=public-rss_20180320</a></span></li></ol>"

In [26]:
def search_compare(querystring, pipeline: "nlp-search-pipeline-equal")
    bm25 = serps(querystring, search_bm25(querystring, show: false), k: 5, show: false)
    knn = serps(querystring, search_knn(querystring, show: false), k: 5, show: false)
    hybrid = serps(querystring, search_hybrid(querystring, pipeline: pipeline, show: false), k: 5, show: false)
    
    html_all = <<-HTML
        <style>
            .compare li {overflow-x:hidden;width:320px!important;text-align:left;height:200px;border-bottom:1px solid #333;}
        </style>
        <table class="compare">
            <tr><td>BM25</td><td>KNN</td><td>Hybrid (#{pipeline})</td></tr>
            <tr><td>#{bm25}</td><td>#{knn}</td><td>#{hybrid}</td></tr>
        </table>
    HTML
    
    IRuby.display(IRuby.html(html_all))
end

:search_compare

In [27]:
search_compare("property market", pipeline: "nlp-search-pipeline-equal")
nil

BM25,KNN,Hybrid (nlp-search-pipeline-equal)
"Showing 697 Results for property marketRICS: Property prices rise(28.265703)Property prices in Cyprus rose in the fourth quarter of 2017, according to RICS' latest Cyprus Property Price Index. Across Cyprus, residential prices for both houses and flats increased on...http://in-cyprus.com/news/business/rics-property-prices-rise/UPDATE: Missing man found on Amamoor property(25.928907)Police searched property late last night.https://www.gympietimes.com.au/news/update-missing-man-found-on-amamoor-property/3212441/Wisconsin property owners appeal ruling in Foxconn lawsuit(24.80427)Wisconsin property owners appeal ruling in Foxconn lawsuithttps://www.taiwannews.com.tw/en/news/3444470Emerging threats in intellectual property crime(24.485075)Experts from across Europe are meeting at the Scottish Police College to discuss the emerging threats in intellectual property crime particularly the illegal use of Internet protocol television (IPTV).http://www.motherwelltimes.co.uk/news/crime/emerging-threats-in-intellectual-property-crime-1-4422203Miami police officer charged with Costa Rica property fraud(23.778606)Miami police officer charged with Costa Rica property fraudhttps://www.taiwannews.com.tw/en/news/3354451","Showing 460 Results for property marketPrices and homes listed up, sales down in Metro Vancouver real estate market(0.8641509)VANCOUVER — The supply of homes in Metro Vancouver’s real estate market hit a three-year high as sales dipped below historical averages last month.The Real Estate Board of Greater Vancouver reports there were 2,425 home sales last month, a 37.7 per cent decline from June 2017.https://nationalpost.com/pmn/news-pmn/canada-news-pmn/prices-and-homes-listed-up-sales-down-in-metro-vancouver-real-estate-marketGiven strong Tampa Bay housing market, is it better to rent or buy?(0.86327225)There's a simple formula to help you determine whether it makes more sense to rent or buy a home in the Tampa Bay area.http://wfla.com/2017/02/15/given-strong-tampa-bay-housing-market-is-it-better-to-rent-or-buy/Feb. home prices soar 6.3 pct in a fierce competition to buy - WAFB 9 News Baton Rouge, Louisiana News, Weather, Sports(0.8600733)http://www.wafb.com/story/38027558/feb-home-prices-soar-63-pct-in-a-fierce-competition-to-buyWhat’s On Listings 31 May-6 Jun(0.85672295)To announce an event or gig, call 01772 838196 or email whatson@lep.co.ukhttps://www.lep.co.uk/whats-on/what-s-on-listings-31-may-6-jun-1-9185417The most ‘ingenious’ real estate deals in New York City(0.8559661)Each spring, commercial real estate pros vie for the Real Estate Board of New York’s Most Ingenious Deal of the Year Awards. For the 73rd annual contest, the...https://nypost.com/2018/03/19/the-most-ingenious-real-estate-deals-in-new-york-city/","Showing 10 Results for property marketPrices and homes listed up, sales down in Metro Vancouver real estate market(0.5)VANCOUVER — The supply of homes in Metro Vancouver’s real estate market hit a three-year high as sales dipped below historical averages last month.The Real Estate Board of Greater Vancouver reports there were 2,425 home sales last month, a 37.7 per cent decline from June 2017.https://nationalpost.com/pmn/news-pmn/canada-news-pmn/prices-and-homes-listed-up-sales-down-in-metro-vancouver-real-estate-marketRICS: Property prices rise(0.5)Property prices in Cyprus rose in the fourth quarter of 2017, according to RICS' latest Cyprus Property Price Index. Across Cyprus, residential prices for both houses and flats increased on...http://in-cyprus.com/news/business/rics-property-prices-rise/Given strong Tampa Bay housing market, is it better to rent or buy?(0.4942914)There's a simple formula to help you determine whether it makes more sense to rent or buy a home in the Tampa Bay area.http://wfla.com/2017/02/15/given-strong-tampa-bay-housing-market-is-it-better-to-rent-or-buy/Feb. home prices soar 6.3 pct in a fierce competition to buy - WAFB 9 News Baton Rouge

In [28]:
search_compare("property market", pipeline: "nlp-search-pipeline-bm25-heavy")
nil

BM25,KNN,Hybrid (nlp-search-pipeline-bm25-heavy)
"Showing 697 Results for property marketRICS: Property prices rise(28.265703)Property prices in Cyprus rose in the fourth quarter of 2017, according to RICS' latest Cyprus Property Price Index. Across Cyprus, residential prices for both houses and flats increased on...http://in-cyprus.com/news/business/rics-property-prices-rise/UPDATE: Missing man found on Amamoor property(25.928907)Police searched property late last night.https://www.gympietimes.com.au/news/update-missing-man-found-on-amamoor-property/3212441/Wisconsin property owners appeal ruling in Foxconn lawsuit(24.80427)Wisconsin property owners appeal ruling in Foxconn lawsuithttps://www.taiwannews.com.tw/en/news/3444470Emerging threats in intellectual property crime(24.485075)Experts from across Europe are meeting at the Scottish Police College to discuss the emerging threats in intellectual property crime particularly the illegal use of Internet protocol television (IPTV).http://www.motherwelltimes.co.uk/news/crime/emerging-threats-in-intellectual-property-crime-1-4422203Miami police officer charged with Costa Rica property fraud(23.778606)Miami police officer charged with Costa Rica property fraudhttps://www.taiwannews.com.tw/en/news/3354451","Showing 460 Results for property marketPrices and homes listed up, sales down in Metro Vancouver real estate market(0.8641509)VANCOUVER — The supply of homes in Metro Vancouver’s real estate market hit a three-year high as sales dipped below historical averages last month.The Real Estate Board of Greater Vancouver reports there were 2,425 home sales last month, a 37.7 per cent decline from June 2017.https://nationalpost.com/pmn/news-pmn/canada-news-pmn/prices-and-homes-listed-up-sales-down-in-metro-vancouver-real-estate-marketGiven strong Tampa Bay housing market, is it better to rent or buy?(0.86327225)There's a simple formula to help you determine whether it makes more sense to rent or buy a home in the Tampa Bay area.http://wfla.com/2017/02/15/given-strong-tampa-bay-housing-market-is-it-better-to-rent-or-buy/Feb. home prices soar 6.3 pct in a fierce competition to buy - WAFB 9 News Baton Rouge, Louisiana News, Weather, Sports(0.8600733)http://www.wafb.com/story/38027558/feb-home-prices-soar-63-pct-in-a-fierce-competition-to-buyWhat’s On Listings 31 May-6 Jun(0.85672295)To announce an event or gig, call 01772 838196 or email whatson@lep.co.ukhttps://www.lep.co.uk/whats-on/what-s-on-listings-31-may-6-jun-1-9185417The most ‘ingenious’ real estate deals in New York City(0.8559661)Each spring, commercial real estate pros vie for the Real Estate Board of New York’s Most Ingenious Deal of the Year Awards. For the 73rd annual contest, the...https://nypost.com/2018/03/19/the-most-ingenious-real-estate-deals-in-new-york-city/","Showing 10 Results for property marketRICS: Property prices rise(0.6)Property prices in Cyprus rose in the fourth quarter of 2017, according to RICS' latest Cyprus Property Price Index. Across Cyprus, residential prices for both houses and flats increased on...http://in-cyprus.com/news/business/rics-property-prices-rise/UPDATE: Missing man found on Amamoor property(0.51420665)Police searched property late last night.https://www.gympietimes.com.au/news/update-missing-man-found-on-amamoor-property/3212441/Wisconsin property owners appeal ruling in Foxconn lawsuit(0.4729166)Wisconsin property owners appeal ruling in Foxconn lawsuithttps://www.taiwannews.com.tw/en/news/3444470Emerging threats in intellectual property crime(0.46119767)Experts from across Europe are meeting at the Scottish Police College to discuss the emerging threats in intellectual property crime particularly the illegal use of Internet protocol television (IPTV).http://www.motherwelltimes.co.uk/news/crime/emerging-threats-in-intellectual-property-crime-1-4422203Miami police officer charged with Costa Rica property fraud(0.43526033)Miami police officer charged with Costa Rica property fraudhttps://

In [29]:
search_compare("property market", pipeline: "nlp-search-pipeline-knn-heavy")
nil

BM25,KNN,Hybrid (nlp-search-pipeline-knn-heavy)
"Showing 697 Results for property marketRICS: Property prices rise(28.265703)Property prices in Cyprus rose in the fourth quarter of 2017, according to RICS' latest Cyprus Property Price Index. Across Cyprus, residential prices for both houses and flats increased on...http://in-cyprus.com/news/business/rics-property-prices-rise/UPDATE: Missing man found on Amamoor property(25.928907)Police searched property late last night.https://www.gympietimes.com.au/news/update-missing-man-found-on-amamoor-property/3212441/Wisconsin property owners appeal ruling in Foxconn lawsuit(24.80427)Wisconsin property owners appeal ruling in Foxconn lawsuithttps://www.taiwannews.com.tw/en/news/3444470Emerging threats in intellectual property crime(24.485075)Experts from across Europe are meeting at the Scottish Police College to discuss the emerging threats in intellectual property crime particularly the illegal use of Internet protocol television (IPTV).http://www.motherwelltimes.co.uk/news/crime/emerging-threats-in-intellectual-property-crime-1-4422203Miami police officer charged with Costa Rica property fraud(23.778606)Miami police officer charged with Costa Rica property fraudhttps://www.taiwannews.com.tw/en/news/3354451","Showing 460 Results for property marketPrices and homes listed up, sales down in Metro Vancouver real estate market(0.8641509)VANCOUVER — The supply of homes in Metro Vancouver’s real estate market hit a three-year high as sales dipped below historical averages last month.The Real Estate Board of Greater Vancouver reports there were 2,425 home sales last month, a 37.7 per cent decline from June 2017.https://nationalpost.com/pmn/news-pmn/canada-news-pmn/prices-and-homes-listed-up-sales-down-in-metro-vancouver-real-estate-marketGiven strong Tampa Bay housing market, is it better to rent or buy?(0.86327225)There's a simple formula to help you determine whether it makes more sense to rent or buy a home in the Tampa Bay area.http://wfla.com/2017/02/15/given-strong-tampa-bay-housing-market-is-it-better-to-rent-or-buy/Feb. home prices soar 6.3 pct in a fierce competition to buy - WAFB 9 News Baton Rouge, Louisiana News, Weather, Sports(0.8600733)http://www.wafb.com/story/38027558/feb-home-prices-soar-63-pct-in-a-fierce-competition-to-buyWhat’s On Listings 31 May-6 Jun(0.85672295)To announce an event or gig, call 01772 838196 or email whatson@lep.co.ukhttps://www.lep.co.uk/whats-on/what-s-on-listings-31-may-6-jun-1-9185417The most ‘ingenious’ real estate deals in New York City(0.8559661)Each spring, commercial real estate pros vie for the Real Estate Board of New York’s Most Ingenious Deal of the Year Awards. For the 73rd annual contest, the...https://nypost.com/2018/03/19/the-most-ingenious-real-estate-deals-in-new-york-city/","Showing 10 Results for property marketPrices and homes listed up, sales down in Metro Vancouver real estate market(0.6)VANCOUVER — The supply of homes in Metro Vancouver’s real estate market hit a three-year high as sales dipped below historical averages last month.The Real Estate Board of Greater Vancouver reports there were 2,425 home sales last month, a 37.7 per cent decline from June 2017.https://nationalpost.com/pmn/news-pmn/canada-news-pmn/prices-and-homes-listed-up-sales-down-in-metro-vancouver-real-estate-marketGiven strong Tampa Bay housing market, is it better to rent or buy?(0.5931497)There's a simple formula to help you determine whether it makes more sense to rent or buy a home in the Tampa Bay area.http://wfla.com/2017/02/15/given-strong-tampa-bay-housing-market-is-it-better-to-rent-or-buy/Feb. home prices soar 6.3 pct in a fierce competition to buy - WAFB 9 News Baton Rouge, Louisiana News, Weather, Sports(0.5682091)http://www.wafb.com/story/38027558/feb-home-prices-soar-63-pct-in-a-fierce-competition-to-buyWhat’s On Listings 31 May-6 Jun(0.5420877)To announce an event or gig, call 01772 838196 or email whatson@lep.co.ukhttps://www.lep.co.uk/whats-

In [30]:
search_compare("crypto scandal")
nil

BM25,KNN,Hybrid (nlp-search-pipeline-equal)
"Showing 164 Results for crypto scandalAP Explains: Why are crypto exchanges vulnerable to hacks?(29.908903)AP Explains: Why are crypto exchanges vulnerable to hacks?https://www.taiwannews.com.tw/en/news/3353987Bitcoin loses more than half its value amid crypto crash(26.140495)After stratospheric growth, Bitcoin crashed below $8,000 for the first time in months amid a continuing global crypto rout.https://www.nbcnews.com/tech/internet/bitcoin-loses-more-half-its-value-amid-crypto-crash-n844056?cid=public-rss_20180202Bitcoin loses more than half its value amid crypto crash(26.140495)After stratospheric growth, Bitcoin crashed below $8,000 for the first time in months amid a continuing global crypto rout.https://www.nbcnews.com/tech/internet/bitcoin-loses-more-half-its-value-amid-crypto-crash-n844056?cid=public-rss_20180204Bitcoin loses more than half its value amid crypto crash(26.140495)After stratospheric growth, Bitcoin crashed below $8,000 for the first time in months amid a continuing global crypto rout.https://www.nbcnews.com/tech/internet/bitcoin-loses-more-half-its-value-amid-crypto-crash-n844056?cid=public-rss_20180203Timeline: Irish Olympic ticket-sale scandal(24.739798)Timeline of events in the Rio Olympics illegal ticket-sale scandal.http://www.bbc.com/news/world-europe-37156930","Showing 460 Results for crypto scandalBitcoin, other cryptocurrencies plunge on wave of bad news(0.8700277)Google said it will ban cryptocurrency-related ads, the IMF called for global regulation and members of Congress slammed them at a hearing Wednesday.https://www.nbcnews.com/tech/tech-news/bitcoin-other-cryptocurrencies-plunge-wave-bad-news-n856821?cid=public-rss_20180320‘Fraud.’ ‘More than a fad.’ The words Wall Street CEOs are using to describe bitcoin(0.8627867)Seems not a day goes by without some Wall Street VIP or financial luminary deciding it’s time for them to weigh in on bitcoin and cryptocurrencies. Here’s a roundup of the latest high-profile statements.http://www.marketwatch.com/story/fraud-or-the-future-how-bitcoin-is-being-described-by-billionaires-and-wall-street-ceos-2017-10-05?siteid=rss&rss=1First U.S. cryptocurrency ban outlaws Venezuela’s ‘Petro’ coin(0.8624979)The U.S. executive branch has issued an executive order prohibiting the use or purchase of Venezuela's Petro cryptocurrency.https://www.digitaltrends.com/computing/us-bans-petro-cryptocurrency/Trump bans US use of Venezuelan cryptocurrency - WAFB 9 News Baton Rouge, Louisiana News, Weather, Sports(0.8622549)http://www.wafb.com/story/37757576/trump-bans-us-use-of-venezuelan-cryptocurrencyEquifax executive hit with insider trading charge for selling stock before data breach went public(0.86135316)A former executive for Equifax is facing criminal and anti-fraud charges after selling stock options before the company went public with the megabreach.https://www.nbcnews.com/business/business-news/equifax-executive-hit-insider-trading-charge-selling-stock-data-breach-n856601?cid=public-rss_20180320","Showing 10 Results for crypto scandalAP Explains: Why are crypto exchanges vulnerable to hacks?(0.5)AP Explains: Why are crypto exchanges vulnerable to hacks?https://www.taiwannews.com.tw/en/news/3353987Bitcoin, other cryptocurrencies plunge on wave of bad news(0.5)Google said it will ban cryptocurrency-related ads, the IMF called for global regulation and members of Congress slammed them at a hearing Wednesday.https://www.nbcnews.com/tech/tech-news/bitcoin-other-cryptocurrencies-plunge-wave-bad-news-n856821?cid=public-rss_20180320‘Fraud.’ ‘More than a fad.’ The words Wall Street CEOs are using to describe bitcoin(0.4570444)Seems not a day goes by without some Wall Street VIP or financial luminary deciding it’s time for them to weigh in on bitcoin and cryptocurrencies. Here’s a roundup of the latest high-profile statements.http://www.marketwatch.com/story/fraud-or-the-future-how-bitcoin-is-being-described-by-billionaires-and-wal

In [31]:
search_compare("US economic recovery")
nil

BM25,KNN,Hybrid (nlp-search-pipeline-equal)
"Showing 2741 Results for US economic recoveryPalestinian protesters disrupt US economic meeting(39.690395)Palestinian protesters disrupt US economic meetinghttps://www.taiwannews.com.tw/en/news/3354307Top EU economic powers warn US about tax plans(34.882103)Top EU economic powers warn US about tax planshttp://abcnews.go.com/International/wireStory/top-eu-economies-warn-us-tax-plans-517190362017: Obaseki assures Edo poeple of speedy economic recovery(33.060505)http://www.nigeriatoday.ng/2017/01/2017-obaseki-assures-edo-poeple-of-speedy-economic-recovery/Building Institutional Capacity Vital for Nigeria’s Economic Recovery, Says Udoma(31.548922)https://www.nigeriatoday.ng/2017/06/building-institutional-capacity-vital-for-nigerias-economic-recovery-says-udoma/Building Institutional Capacity Vital for Nigeria’s Economic Recovery, Says Udoma(31.548922)http://www.nigeriatoday.ng/2017/06/building-institutional-capacity-vital-for-nigerias-economic-recovery-says-udoma/","Showing 460 Results for US economic recoveryUS consumer confidence rebounds in April - WAFB 9 News Baton Rouge, Louisiana News, Weather, Sports(0.8782718)http://www.wafb.com/story/38028414/us-consumer-confidence-rebounds-in-aprilUS factory output rises for 2nd straight month in latest sign of manufacturing rebound(0.87431824)US factory output rises for 2nd straight month in latest sign of manufacturing reboundhttp://www.taiwannews.com.tw/en/news/3095825Trends Shaping The Return of Manufacturing to US(0.87235814)http://www.industryweek.com/global-economy/trends-shaping-return-manufacturing-usUpbeat ‘Beige Book’ prognosis of economy keeps Fed on track for June rate hike(0.8701608)The Federal Reserve said the U.S. grew “moderately” from late April to early May in its latest evaluation of the economy, indicating the central bank remains firmly on track to raise interest rates next month.https://www.marketwatch.com/story/upbeat-beige-book-prognosis-of-economy-keeps-fed-on-track-for-june-rate-hike-2018-05-30?siteid=rss&rss=1US businesses increase inventories 0.4 percent in December, biggest sales gain since 2011(0.86628675)US businesses increase inventories 0.4 percent in December, biggest sales gain since 2011http://www.taiwannews.com.tw/en/news/3095845","Showing 10 Results for US economic recovery2017: Obaseki assures Edo poeple of speedy economic recovery(0.83361274)http://www.nigeriatoday.ng/2017/01/2017-obaseki-assures-edo-poeple-of-speedy-economic-recovery/US consumer confidence rebounds in April - WAFB 9 News Baton Rouge, Louisiana News, Weather, Sports(0.5)http://www.wafb.com/story/38028414/us-consumer-confidence-rebounds-in-aprilPalestinian protesters disrupt US economic meeting(0.5)Palestinian protesters disrupt US economic meetinghttps://www.taiwannews.com.tw/en/news/3354307US factory output rises for 2nd straight month in latest sign of manufacturing rebound(0.48007944)US factory output rises for 2nd straight month in latest sign of manufacturing reboundhttp://www.taiwannews.com.tw/en/news/3095825Trends Shaping The Return of Manufacturing to US(0.47020325)http://www.industryweek.com/global-economy/trends-shaping-return-manufacturing-us"
